# CSV DataSource Testing Notebook

This notebook demonstrates how to use the unified DataSource abstraction to read and operate on CSV files.

In [2]:
# Setup: Add project root to path
import sys
sys.path.insert(0, '..')

# Core imports
from src.datasource import create_datasource, DataSourceFactory, CSVDataSource
from src.datasource.base import SourceType, TableInfo, ColumnInfo, RelationshipInfo

# For tools
from src.tools import (
    register_datasource,
    get_datasource,
    clear_registry,
    get_dataset_overview,
    get_table_info,
    get_row_count,
    get_column_names,
    get_column_statistics,
    get_sample_rows,
    find_common_columns,
    get_relationships,
)

import pandas as pd
from pprint import pprint

print("Imports successful!")

Imports successful!


## 1. Single CSV File

In [3]:
# Create DataSource from a single CSV file
single_ds = create_datasource('../data/biota_dataset/biota.csv')

print(f"Name: {single_ds.name}")
print(f"Type: {single_ds.source_type}")
print(f"Tables: {single_ds.tables}")
print(f"Multi-table: {single_ds.is_multi_table}")

Name: dataset
Type: SourceType.CSV
Tables: ['biota']
Multi-table: False


In [4]:
# Get table info
table_name = single_ds.tables[0]
info = single_ds.get_table_info(table_name)

print(f"Table: {info.name}")
print(f"Rows: {info.row_count}")
print(f"Columns: {info.column_count}")
print(f"\nColumn details:")
for col in info.columns:
    print(f"  - {col.name}: {col.dtype}")

Table: biota
Rows: 391018
Columns: 4

Column details:
  - sample_id: int64
  - sibes_id: int64
  - abundance_m2: float64
  - afdm_m2: float64


In [5]:
# Read the table as DataFrame
df = single_ds.read_table(table_name, nrows=10)
df

,sample_id,sibes_id,abundance_m2,afdm_m2
0,33941,16,28.8716,0.4331
1,33941,20,115.4866,0.6986
2,33941,21,115.4866,0.0000
3,33941,289,230.9732,0.0462
4,33942,20,288.7165,0.7391
5,33942,26,57.7433,0.0115
6,33942,289,404.2030,0.0808
7,33943,14,202.1015,2.1942
8,33943,16,86.6149,1.2473
9,33943,20,346.4597,0.4735


## 2. Multiple CSV Files (Multi-table Dataset)

In [7]:
# Create DataSource from multiple CSV files using a dictionary
multi_ds = create_datasource(
    {
        'biota': '../data/biota_dataset/biota.csv',
        'samples': '../data/biota_dataset/samples.csv',
        'species': '../data/biota_dataset/species.csv',
    },
    name='biota_study'
)

print(f"Name: {multi_ds.name}")
print(f"Type: {multi_ds.source_type}")
print(f"Tables: {multi_ds.tables}")
print(f"Multi-table: {multi_ds.is_multi_table}")

Name: biota_study
Type: SourceType.CSV
Tables: ['biota', 'samples', 'species']
Multi-table: True


In [8]:
# Get info for all tables
for table in multi_ds.tables:
    info = multi_ds.get_table_info(table)
    print(f"\n{table}:")
    print(f"  Rows: {info.row_count}")
    print(f"  Columns: {info.column_names}")


biota:
  Rows: 391018
  Columns: ['sample_id', 'sibes_id', 'abundance_m2', 'afdm_m2']

samples:
  Rows: 51851
  Columns: ['sample_id', 'sampling_station_id', 'sampling_type', 'date', 'platform', 'tidal_basin_name', 'tidal_flat_name', 'x', 'y', 'median_grain_size', 'percentage_mud']

species:
  Rows: 177
  Columns: ['sibes_id', 'name', 'short_name', 'aphia_id', 'taxonomic_group', 'taxonomic_indentification_level', 'year_added', 'occurance', 'occupancy', 'weight_is_measured', 'length_measuring_method_id.', 'min_shell_length_to_separate_flesh', 'missing_afdm_method_id..', 'remarks']


In [9]:
# Preview each table
for table in multi_ds.tables:
    print(f"\n--- {table} (first 3 rows) ---")
    display(multi_ds.read_table(table, nrows=3))


--- biota (first 3 rows) ---


,sample_id,sibes_id,abundance_m2,afdm_m2
0,33941,16,28.8716,0.4331
1,33941,20,115.4866,0.6986
2,33941,21,115.4866,0.0000



--- samples (first 3 rows) ---


,sample_id,sampling_station_id,sampling_type,date,platform,tidal_basin_name,tidal_flat_name,x,y,median_grain_size,percentage_mud
0,33941,59,grid,2008-07-14,boat,Marsdiep,Vlakte van Kerken,4.902667,53.089333,NaN,NaN
1,33942,456,grid,2008-07-14,boat,Eierlandse Gat,Vlakte van Kerken,4.972500,53.134667,166.5,2.77
2,33943,457,grid,2008-07-14,boat,Eierlandse Gat,Vlakte van Kerken,4.980333,53.134667,NaN,NaN



--- species (first 3 rows) ---


,sibes_id,name,short_name,aphia_id,taxonomic_group,taxonomic_indentification_level,year_added,occurance,occupancy,weight_is_measured,length_measuring_method_id.,min_shell_length_to_separate_flesh,missing_afdm_method_id..,remarks
0,57,Abra alba,Abralb,141433,bivalve,species,2000,397,0.8,True,6,8,12.0,NaN
1,58,Abra tenuis,Abrten,141439,bivalve,species,2000,1898,3.7,True,6,8,24.0,NaN
2,261,Bivalvia sp.,Bivspe,105,bivalve,class,2017,609,1.2,True,6,8,NaN,Parts of individuals.


In [10]:
# Create DataSource from a directory containing CSV files
dir_ds = create_datasource('../data/biota_dataset/', name='biota_from_dir')

print(f"Name: {dir_ds.name}")
print(f"Type: {dir_ds.source_type}")
print(f"Tables: {dir_ds.tables}")
print(f"Multi-table: {dir_ds.is_multi_table}")

Name: biota_from_dir
Type: SourceType.CSV
Tables: ['species', 'samples', 'biota']
Multi-table: True


## 4. Relationship Discovery

In [11]:
# Discover relationships between tables
relationships = multi_ds.get_relationships()

print(f"Found {len(relationships)} relationship(s):\n")
for rel in relationships:
    print(f"  {rel.from_table}.{rel.from_column} -> {rel.to_table}.{rel.to_column}")
    print(f"    Type: {rel.relationship_type}")
    print(f"    Confidence: {rel.confidence:.2f}")
    print()

Found 2 relationship(s):

  biota.sample_id -> samples.sample_id
    Type: many-to-one
    Confidence: 1.00

  biota.sibes_id -> species.sibes_id
    Type: many-to-one
    Confidence: 1.00



## 5. Get Full Schema

In [ ]:
# Get the complete schema
schema = multi_ds.get_schema()
pprint(schema)

## 6. Using DataSource Tools

Tools are used by the AI players to analyze data. They work with registered DataSources.

In [ ]:
# Clear any previous registrations and register our DataSource
clear_registry()
register_datasource('my_ds', multi_ds)
print("DataSource registered as 'my_ds'")

In [ ]:
# Get dataset overview (what players see first)
overview = get_dataset_overview.invoke({'datasource_key': 'my_ds'})
pprint(overview)

In [ ]:
# Get info for a specific table
biota_info = get_table_info.invoke({'datasource_key': 'my_ds', 'table': 'biota'})
pprint(biota_info)

In [ ]:
# Get column statistics
stats = get_column_statistics.invoke({'datasource_key': 'my_ds', 'table': 'biota'})
pprint(stats)

In [ ]:
# Get sample rows
samples = get_sample_rows.invoke({'datasource_key': 'my_ds', 'table': 'samples', 'n': 5})
print(samples)

In [ ]:
# Find common columns across tables (potential join keys)
common = find_common_columns.invoke({'datasource_key': 'my_ds'})
pprint(common)

In [ ]:
# Get relationships via tool
rels = get_relationships.invoke({'datasource_key': 'my_ds'})
pprint(rels)

## 7. Chunked Reading (Large Files)

In [ ]:
# For large files, use iter_table to process in chunks
chunk_count = 0
total_rows = 0

for chunk in multi_ds.iter_table('biota', chunksize=100):
    chunk_count += 1
    total_rows += len(chunk)
    print(f"Chunk {chunk_count}: {len(chunk)} rows")

print(f"\nTotal: {total_rows} rows in {chunk_count} chunks")

## 8. Direct DataFrame Operations

In [ ]:
# Read specific columns only
df = multi_ds.read_table('biota', columns=['sample_id', 'species_code'] if 'species_code' in multi_ds.get_table_info('biota').column_names else None)
print(f"Columns read: {df.columns.tolist()}")
df.head()

In [ ]:
# Get unique values from a column
first_col = multi_ds.get_table_info('samples').column_names[0]
unique_vals = multi_ds.get_column_values('samples', first_col, limit=10)
print(f"First 10 unique values in '{first_col}': {unique_vals}")

## 9. DataSource Serialization

In [ ]:
# DataSource can be serialized to dict (useful for passing to tools)
ds_dict = multi_ds.to_dict()
pprint(ds_dict)

# String representation
print(f"\nrepr: {repr(multi_ds)}")
print(f"str:  {str(multi_ds)}")

## Summary

The DataSource abstraction provides:

1. **Unified Interface**: Same API for single files, multiple files, directories
2. **Lazy Loading**: Data loaded only when accessed
3. **Caching**: Table info and relationships are cached
4. **Relationship Discovery**: Automatic detection of foreign keys
5. **Tool Integration**: Works with the tools used by AI players
6. **Chunked Reading**: Handle large files with `iter_table()`